# JSON 구조 재구성: 평면 구조 → 계층 구조

## 목적
- title 필드에 모든 내용이 들어있는 섹션(3, 4번 등)을 1, 2번처럼 계층 구조로 재구성
- Lab-scale: design_standards_db_example.json
- Pilot-scale: design_standards_db.json (나중에 적용)

In [5]:
import json
import re
from typing import Dict, List, Any

## 1. 원본 파일 로드

In [6]:
# Lab-scale 파일 경로
input_file = r'd:\DATA\RAG_AGENT\design_standards_db_example.json'
output_file = r'd:\DATA\RAG_AGENT\design_standards_db_example_restructured.json'

# 파일 로드
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"총 {len(data)}개의 최상위 섹션 로드됨")
for item in data[:5]:
    print(f"- Code: {item['code']}, Title: {item['title'][:50]}...")

총 4개의 최상위 섹션 로드됨
- Code: 1, Title: 총설...
- Code: 2, Title: 착수정...
- Code: 3, Title: 응집용 약품주입설비\n\n3.1 총칙\n\n급속여과방식의 정수처리에서는 전처리로서 약품에 ...
- Code: 4, Title: 응집지\n\n4.1 총칙\n\n원수의 탁질 중에서 입경이 10-2mm 이상인 것은 보통 침...


## 2. 구조 분석 함수

In [7]:
def analyze_section_structure(section: Dict) -> Dict[str, Any]:
    """
    섹션의 구조를 분석합니다.
    - has_children: children 배열에 실제 하위 섹션이 있는지
    - content_in_title: title에 하위 섹션 내용이 포함되어 있는지
    """
    has_children = len(section.get('children', [])) > 0
    title = section.get('title', '')
    content = section.get('content', '')
    
    # title에 하위 섹션 코드 패턴이 있는지 확인 (예: 3.1, 3.2)
    code = section.get('code', '')
    pattern = rf'{re.escape(code)}\.\d+'
    content_in_title = bool(re.search(pattern, title))
    
    return {
        'code': code,
        'has_children': has_children,
        'content_in_title': content_in_title,
        'content_length': len(content),
        'title_length': len(title),
        'needs_restructure': content_in_title and not has_children
    }

# 전체 데이터 분석
print("섹션별 구조 분석:")
print("=" * 80)
for section in data:
    analysis = analyze_section_structure(section)
    if analysis['needs_restructure']:
        print(f"⚠️  Code {analysis['code']}: 재구성 필요!")
    else:
        print(f"✓  Code {analysis['code']}: 정상 구조")
    print(f"   - Children: {analysis['has_children']}, "
          f"Content in Title: {analysis['content_in_title']}, "
          f"Title Length: {analysis['title_length']}")
    print()

섹션별 구조 분석:
✓  Code 1: 정상 구조
   - Children: True, Content in Title: False, Title Length: 2

✓  Code 2: 정상 구조
   - Children: True, Content in Title: False, Title Length: 3

⚠️  Code 3: 재구성 필요!
   - Children: False, Content in Title: True, Title Length: 2742

⚠️  Code 4: 재구성 필요!
   - Children: False, Content in Title: True, Title Length: 1709



## 3. 텍스트 파싱 함수

In [8]:
def parse_hierarchical_content(text: str, parent_code: str) -> List[Dict[str, Any]]:
    """
    평면 텍스트를 계층 구조로 파싱합니다.
    
    예시 입력:
    3.1 총칙\n\n내용...\n\n3.2 응집제\n\n내용...
    
    예시 출력:
    [
        {"code": "3.1", "title": "총칙", "content": "내용...", "children": []},
        {"code": "3.2", "title": "응집제", "content": "내용...", "children": []}
    ]
    """
    # 섹션 헤더 패턴: "3.1 제목" 형태
    pattern = rf'^({re.escape(parent_code)}\.\d+(?:\.\d+)*)\s+(.+?)$'
    
    lines = text.split('\n')
    sections = []
    current_section = None
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # 섹션 헤더 매칭
        match = re.match(pattern, line)
        if match:
            # 이전 섹션 저장
            if current_section:
                sections.append(current_section)
            
            # 새 섹션 시작
            code = match.group(1)
            title = match.group(2).strip()
            current_section = {
                'code': code,
                'title': title,
                'content': '',
                'children': []
            }
        elif current_section:
            # 현재 섹션의 내용에 추가
            if current_section['content']:
                current_section['content'] += '\n' + line
            else:
                current_section['content'] = line
    
    # 마지막 섹션 저장
    if current_section:
        sections.append(current_section)
    
    return sections

# 테스트
test_text = """3.1 총칙

급속여과방식의 정수처리에서는 전처리로서 약품에 의한 응집이 필수적이다.

3.2 응집제

(1) 응집제의 종류는 원수의 수량...
"""

parsed = parse_hierarchical_content(test_text, "3")
print("파싱 테스트 결과:")
for section in parsed:
    print(f"Code: {section['code']}, Title: {section['title']}")
    print(f"Content: {section['content'][:50]}...")
    print()

파싱 테스트 결과:
Code: 3.1, Title: 총칙
Content: 급속여과방식의 정수처리에서는 전처리로서 약품에 의한 응집이 필수적이다....

Code: 3.2, Title: 응집제
Content: (1) 응집제의 종류는 원수의 수량......



## 4. 계층 구조 재구성 함수

In [9]:
def build_hierarchy(sections: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    평면 섹션 리스트를 계층 구조로 변환합니다.
    
    예: [3.1, 3.2, 3.2.1, 3.2.2, 3.3] -> 계층 구조
    """
    if not sections:
        return []
    
    # 코드 레벨 계산 (점의 개수)
    def get_level(code: str) -> int:
        return code.count('.')
    
    # 부모 코드 찾기
    def get_parent_code(code: str) -> str:
        parts = code.rsplit('.', 1)
        return parts[0] if len(parts) > 1 else None
    
    # 섹션 딕셔너리 생성 (빠른 검색용)
    section_dict = {s['code']: s for s in sections}
    
    # 최상위 레벨 찾기
    root_level = min(get_level(s['code']) for s in sections)
    root_sections = [s for s in sections if get_level(s['code']) == root_level]
    
    # 각 섹션을 부모에 연결
    for section in sections:
        parent_code = get_parent_code(section['code'])
        if parent_code and parent_code in section_dict:
            parent = section_dict[parent_code]
            if section not in parent['children']:
                parent['children'].append(section)
    
    return root_sections

# 테스트
test_sections = [
    {'code': '3.1', 'title': '총칙', 'content': 'A', 'children': []},
    {'code': '3.2', 'title': '응집제', 'content': 'B', 'children': []},
    {'code': '3.2.1', 'title': '종류', 'content': 'C', 'children': []},
    {'code': '3.3', 'title': 'pH조정제', 'content': 'D', 'children': []},
]

hierarchical = build_hierarchy(test_sections)
print("계층 구조 빌드 테스트:")
print(json.dumps(hierarchical, indent=2, ensure_ascii=False)[:500])

계층 구조 빌드 테스트:
[
  {
    "code": "3.1",
    "title": "총칙",
    "content": "A",
    "children": []
  },
  {
    "code": "3.2",
    "title": "응집제",
    "content": "B",
    "children": [
      {
        "code": "3.2.1",
        "title": "종류",
        "content": "C",
        "children": []
      }
    ]
  },
  {
    "code": "3.3",
    "title": "pH조정제",
    "content": "D",
    "children": []
  }
]


## 5. 메인 재구성 함수

In [10]:
def restructure_section(section: Dict[str, Any]) -> Dict[str, Any]:
    """
    단일 섹션을 재구성합니다.
    """
    analysis = analyze_section_structure(section)
    
    if not analysis['needs_restructure']:
        # 재구성 불필요 - 원본 반환 (하위 섹션은 재귀적으로 처리)
        return {
            'code': section['code'],
            'title': section['title'],
            'content': section['content'],
            'children': [restructure_section(child) for child in section.get('children', [])]
        }
    
    # 재구성 필요
    print(f"재구성 중: Code {section['code']}")
    
    # title에서 실제 제목과 본문 분리
    title_text = section['title']
    lines = title_text.split('\n')
    
    # 첫 번째 줄이 제목
    actual_title = lines[0].strip()
    
    # 나머지는 본문
    remaining_text = '\n'.join(lines[1:]).strip()
    
    # 하위 섹션 파싱
    parsed_children = parse_hierarchical_content(remaining_text, section['code'])
    
    # 계층 구조 빌드
    hierarchical_children = build_hierarchy(parsed_children)
    
    # 본문 내용 추출 (하위 섹션 전 내용)
    if parsed_children:
        first_child_pattern = rf'{re.escape(parsed_children[0]["code"])}\s+{re.escape(parsed_children[0]["title"])}'
        match = re.search(first_child_pattern, remaining_text)
        if match:
            main_content = remaining_text[:match.start()].strip()
        else:
            main_content = section.get('content', '')
    else:
        main_content = remaining_text
    
    return {
        'code': section['code'],
        'title': actual_title,
        'content': main_content,
        'children': hierarchical_children
    }

def restructure_all(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    전체 데이터를 재구성합니다.
    """
    print("=" * 80)
    print("재구성 시작...")
    print("=" * 80)
    
    restructured = []
    for section in data:
        restructured_section = restructure_section(section)
        restructured.append(restructured_section)
    
    print("=" * 80)
    print("재구성 완료!")
    print("=" * 80)
    
    return restructured

## 6. 재구성 실행

In [ ]:
# 재구성 실행
restructured_data = restructure_all(data)

## 7. 결과 검증

In [ ]:
def print_structure(section: Dict[str, Any], indent: int = 0):
    """
    섹션 구조를 트리 형태로 출력합니다.
    """
    prefix = "  " * indent
    title_preview = section['title'][:50] + "..." if len(section['title']) > 50 else section['title']
    content_len = len(section['content'])
    children_count = len(section['children'])
    
    print(f"{prefix}[{section['code']}] {title_preview}")
    print(f"{prefix}    Content: {content_len} chars, Children: {children_count}")
    
    for child in section['children']:
        print_structure(child, indent + 1)

print("재구성된 데이터 구조:")
print("=" * 80)
for section in restructured_data:
    print_structure(section)
    print()

In [ ]:
# 특정 섹션 상세 확인 (예: 코드 3번)
section_3 = next((s for s in restructured_data if s['code'] == '3'), None)
if section_3:
    print("섹션 3 상세 정보:")
    print("=" * 80)
    print(f"Code: {section_3['code']}")
    print(f"Title: {section_3['title']}")
    print(f"Content Length: {len(section_3['content'])}")
    print(f"Children Count: {len(section_3['children'])}")
    print(f"\nChildren:")
    for child in section_3['children']:
        print(f"  - [{child['code']}] {child['title']}")

## 8. 결과 저장

In [ ]:
# JSON 파일로 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(restructured_data, f, ensure_ascii=False, indent=4)

print(f"✓ 재구성된 데이터가 저장되었습니다: {output_file}")

# 파일 크기 비교
import os
original_size = os.path.getsize(input_file)
new_size = os.path.getsize(output_file)

print(f"\n파일 크기 비교:")
print(f"원본: {original_size:,} bytes")
print(f"재구성: {new_size:,} bytes")
print(f"차이: {new_size - original_size:+,} bytes ({(new_size/original_size - 1)*100:+.1f}%)")

## 9. Pilot Scale 적용 준비

In [ ]:
# Pilot scale (큰 파일)에 적용하려면 아래 주석을 해제하고 실행

# pilot_input = r'd:\DATA\RAG_AGENT\design_standards_db.json'
# pilot_output = r'd:\DATA\RAG_AGENT\design_standards_db_restructured.json'

# print("Pilot scale 파일 처리 시작...")
# with open(pilot_input, 'r', encoding='utf-8') as f:
#     pilot_data = json.load(f)

# pilot_restructured = restructure_all(pilot_data)

# with open(pilot_output, 'w', encoding='utf-8') as f:
#     json.dump(pilot_restructured, f, ensure_ascii=False, indent=4)

# print(f"✓ Pilot scale 처리 완료: {pilot_output}")

## 10. 요약 통계

In [ ]:
def count_sections(data: List[Dict[str, Any]]) -> Dict[str, int]:
    """
    전체 섹션 수를 재귀적으로 계산합니다.
    """
    total = 0
    by_level = {}
    
    def count_recursive(sections, level=1):
        nonlocal total
        count = len(sections)
        total += count
        by_level[level] = by_level.get(level, 0) + count
        
        for section in sections:
            if section['children']:
                count_recursive(section['children'], level + 1)
    
    count_recursive(data)
    return {'total': total, 'by_level': by_level}

original_stats = count_sections(data)
restructured_stats = count_sections(restructured_data)

print("통계 비교:")
print("=" * 80)
print(f"원본 섹션 수: {original_stats['total']}")
print(f"재구성 섹션 수: {restructured_stats['total']}")
print(f"\n레벨별 섹션 수 (재구성 후):")
for level, count in sorted(restructured_stats['by_level'].items()):
    print(f"  Level {level}: {count}개")